In [1]:
import sys
import os
from PIL import Image, ImageOps
from multiprocessing import cpu_count
import threading
import queue

In [2]:
Larger_Ratio = 3
Tile_Size = 50
Target_Path = './target1.jpg'
Tile_Path = './cat'

In [3]:
class TargetProcess:
    def __init__(self, targetpath):
        self.path = targetpath
    def getTarget(self):
        print("try to open the target image.")
        img = Image.open(self.path)
        width = img.size[0]*Larger_Ratio
        height = img.size[1]*Larger_Ratio
        larger_img = img.resize((width, height), Image.ANTIALIAS)
        #裁切無法整除的部分
        margin_w = (width % Tile_Size)/2
        margin_h = (height % Tile_Size)/2
        if margin_w >0 or margin_h>0:
            large_img = larger_img.crop((margin_w, margin_h, width - margin_w, height- margin_h))
        image_data = large_img.convert('RGB')
        
        print("open correct and get the bigger image")
        return image_data

In [4]:
class TileProcess:
    def __init__(self, tilepath):
        self.path = tilepath
    
    def process_tile(self, tile_path):
        try:
            img = Image.open(tile_path)
            width = img.size[0]
            height = img.size[1]
            min_length  = min(width, height)
            w_crop = (width - min_length) / 2
            h_crop = (height - min_length) / 2
            img = img.crop((w_crop, h_crop, width - w_crop, height - h_crop))
            tile_img = img.resize((Tile_Size, Tile_Size), Image.ANTIALIAS)
            return  tile_img.convert('RGB')
        except:
            return None
    
    def get_tiles(self):
        tiles = []

        print('Reading tiles from {}...'.format(self.path))
        # search the tiles directory recursively
        for root, subFolders, files in os.walk(self.path):
            for tile_name in files:
                print('Reading {:40.40}'.format(tile_name))

                tile_path = os.path.join(root, tile_name)
                tile  = self.process_tile(tile_path)
                if tile != None:
                    tiles.append(tile)

        print('Processed {} tiles.'.format(len(tiles)))
        return tiles

In [5]:
class MosaicImage:
    def __init__(self, original_img):
        self.image = Image.new(original_img.mode, original_img.size)
        self.x_tile_count = int(original_img.size[0] / Tile_Size)
        self.y_tile_count = int(original_img.size[1] / Tile_Size)
        self.total_tiles  = self.x_tile_count * self.y_tile_count

    def add_tile(self, tile_data, coords):
        img = Image.new('RGB', (Tile_Size, Tile_Size))
        img.putdata(tile_data)
        self.image.paste(img, coords)

    def save(self, path):
        self.image.save(path)

In [6]:

class TileFitter:
    def __init__(self, tiles_data):
        self.tiles_data = tiles_data

    def __get_tile_diff(self, t1, t2, bail_out_value):
        diff = 0
        for i in range(len(t1)):
            #diff += (abs(t1[i][0] - t2[i][0]) + abs(t1[i][1] - t2[i][1]) + abs(t1[i][2] - t2[i][2]))
            diff += ((t1[i][0] - t2[i][0])**2 + (t1[i][1] - t2[i][1])**2 + (t1[i][2] - t2[i][2])**2)
        return diff
    def get_best_fit_tile(self, img_data):
        best_fit_tile_index = None
        min_diff = sys.maxsize
        tile_index = 0

        # go through each tile in turn looking for the best match for the part of the image represented by 'img_data'
        for tile_data in self.tiles_data:
            diff = self.__get_tile_diff(img_data, tile_data.getdata(), min_diff)
            if diff < min_diff:
                min_diff = diff
                best_fit_tile_index = tile_index
            tile_index += 1

        return best_fit_tile_index

In [7]:
def Generate_Mosaic(target, tiles):
    print("start to generate the mosaic image ：）")
    New_Balnk_Image = MosaicImage(target)
    tile_function = TileFitter(tiles)
    all_tile  =  [list(tile.getdata()) for tile in tiles]
    count = 0 
    for x in range(New_Balnk_Image.x_tile_count):
        for y in range(New_Balnk_Image.y_tile_count):
            boxCoord = (x * Tile_Size, y * Tile_Size, (x + 1) * Tile_Size, (y + 1) * Tile_Size)
            index = tile_function.get_best_fit_tile( target.crop(boxCoord).getdata())
            New_Balnk_Image.add_tile(tiles[index].getdata(), boxCoord)
            count +=1
            print("It is processing on "+str(count)+'/'+str(New_Balnk_Image.x_tile_count*New_Balnk_Image.y_tile_count))
    New_Balnk_Image.save('./here.jpg')

In [8]:
# class Worker(threading.Thread):
#     def __init__(self, queue, num, lock):
#         threading.Thread.__init__(self)
#         self.queue = queue
#         self.num = num
#         self.lock = lock

#     def run(self):
#         while self.queue.qsize() > 0:
#             process_num = self.queue.get()
#             x =  self.num % New_Balnk_Image.x_tile_count
#             y = int(  self.num / New_Balnk_Image.x_tile_count)
#             boxCoord = (x * Tile_Size, y * Tile_Size, (x + 1) * Tile_Size, (y + 1) * Tile_Size)
#             index = tile_function.get_best_fit_tile( target.crop(boxCoord).getdata())
#             # 取得 lock
#             self.lock.acquire()
#             print("Lock acquired by Worker %d" % self.num)
                
#             # 不能讓多個執行緒同時進的工作
#             print("Worker %d: %s" % (self.num, process_num))
#             New_Balnk_Image.add_tile(tiles[index].getdata(), boxCoord)
#             # 釋放 lock
#             print("Lock released by Worker %d" % self.num)
#             self.lock.release()
            
# my_queue = queue.Queue()
# for i in range(New_Balnk_Image.x_tile_count*New_Balnk_Image.y_tile_count):
#     my_queue.put(i)

# lock = threading.Lock()
# workers = []
# for num in range(int(cpu_count())):
#     workers.append( Worker(my_queue, num, lock))
    
# for worker in workers:
#     worker.start()


def Multi_processing_Generate_Mosaic(target, tiles):
    print("start to generate the mosaic image ：）")
    New_Balnk_Image = MosaicImage(target)
    tile_function = TileFitter(tiles)
    all_tile  =  [list(tile.getdata()) for tile in tiles]
    
    class Worker(threading.Thread):
        def __init__(self, queue, num, lock):
            threading.Thread.__init__(self)
            self.queue = queue
            self.num = num
            self.lock = lock

        def run(self):
            while self.queue.qsize() > 0:
                process_num = self.queue.get()
                x =  self.num % New_Balnk_Image.x_tile_count
                y = int(  self.num / New_Balnk_Image.x_tile_count)
                boxCoord = (x * Tile_Size, y * Tile_Size, (x + 1) * Tile_Size, (y + 1) * Tile_Size)
                index = tile_function.get_best_fit_tile( target.crop(boxCoord).getdata())
                # 取得 lock
#                 self.lock.acquire()
#                 print("Lock acquired by Worker %d" % self.num)
                # 不能讓多個執行緒同時進的工作
                print("Worker %d: %s" % (self.num, process_num))
                New_Balnk_Image.add_tile(tiles[index].getdata(), boxCoord)
                # 釋放 lock
#                 print("Lock released by Worker %d" % self.num)
#                 self.lock.release()
    print("可用CPU數量： ", cpu_count())
    my_queue = queue.Queue()
    for i in range(New_Balnk_Image.x_tile_count*New_Balnk_Image.y_tile_count):
        my_queue.put(i)

    lock = threading.Lock()
    workers = []
    for num in range(int(cpu_count())):
        workers.append( Worker(my_queue, num, lock))

    for worker in workers:
        worker.start()
    for worker in workers:
        worker.join()
    print("Done")
    New_Balnk_Image.save('./here2.jpg')

In [9]:
import time
time_start = time.time() #開始計時
#man
big = TargetProcess(Target_Path).getTarget()
tiles = TileProcess(Tile_Path).get_tiles()
# Generate_Mosaic(big,tiles)
Multi_processing_Generate_Mosaic(big,tiles)

time_end = time.time()    #結束計時
time_c= time_end - time_start   #執行所花時間
print('time cost', time_c, 's')

try to open the target image.
open correct and get the bigger image
Reading tiles from ./cat...
Reading cat191.jpeg                             
Reading cat257.jpeg                             
Reading cat312.jpeg                             
Reading cat336.png                              
Reading cat129.jpeg                             
Reading cat083.jpeg                             
Reading cat200.jpeg                             
Reading cat345.jpeg                             
Reading cat095.jpeg                             
Reading cat216.jpeg                             
Reading cat353.jpeg                             
Reading cat187.jpeg                             
Reading cat168.jpeg                             
Reading cat241.jpeg                             
Reading cat304.jpeg                             
Reading cat113.jpeg                             
Reading cat056.jpeg                             
Reading cat390.jpeg                             
Reading cat144.jpeg   

Reading cat068.jpeg                             
Reading cat316.jpeg                             
Reading cat253.jpeg                             
Reading cat195.jpeg                             
Reading cat194.jpeg                             
Reading cat317.jpeg                             
Reading cat252.jpeg                             
Reading cat086.jpeg                             
Reading cat069.jpeg                             
Reading cat340.jpeg                             
Reading cat205.jpeg                             
Reading cat090.jpeg                             
Reading cat356.jpeg                             
Reading cat213.jpeg                             
Reading cat028.jpeg                             
Reading cat182.jpeg                             
Reading cat301.jpeg                             
Reading cat244.jpeg                             
Reading cat053.jpeg                             
Reading cat116.jpeg                             
Reading cat395.jpeg 

Worker 1: 61
Worker 0: 62
Worker 3: 63
Worker 2: 64
Worker 0: 66
Worker 1: 65
Worker 3: 67
Worker 2: 68
Worker 0: 69
Worker 1: 70
Worker 3: 71
Worker 2: 72
Worker 0: 73
Worker 1: 74
Worker 3: 75
Worker 2: 76
Worker 0: 77Worker 1: 78

Worker 3: 79
Worker 2: 80
Worker 1: 82
Worker 0: 81
Worker 3: 83
Worker 2: 84
Worker 1: 85
Worker 0: 86
Worker 3: 87
Worker 2: 88
Worker 1: 89
Worker 0: 90
Worker 3: 91
Worker 2: 92
Worker 1: 93
Worker 0: 94
Worker 3: 95
Worker 2: 96
Worker 1: 97
Worker 0: 98
Worker 3: 99
Worker 2: 100
Worker 1: 101
Worker 0: 102
Worker 3: 103
Worker 2: 104
Worker 1: 105
Worker 0: 106
Worker 3: 107
Worker 2: 108
Worker 1: 109
Worker 0: 110
Worker 3: 111
Worker 2: 112
Worker 1: 113
Worker 0: 114
Worker 3: 115
Worker 2: 116
Worker 1: 117
Worker 0: 118
Worker 3: 119
Worker 2: 120
Worker 1: 121
Worker 0: 122
Worker 3: 123
Worker 2: 124
Worker 1: 125
Worker 3: 127
Worker 0: 126
Worker 2: 128
Worker 1: 129
Worker 0: 131
Worker 3: 130
Worker 2: 132
Worker 1: 133
Worker 0: 134
Wor

In [15]:

print(cpu_count())

4
